# Geoqueries en Mongo

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#¿Qué-podemos-hacer-con-las-geoqueries?" data-toc-modified-id="¿Qué-podemos-hacer-con-las-geoqueries?-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>¿Qué podemos hacer con las geoqueries?</a></span></li><li><span><a href="#Generamos-la-conexión-con-Mongo" data-toc-modified-id="Generamos-la-conexión-con-Mongo-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Generamos la conexión con Mongo</a></span></li><li><span><a href="#Importar-colecciones" data-toc-modified-id="Importar-colecciones-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Importar colecciones</a></span></li><li><span><a href="#Crear-indexes" data-toc-modified-id="Crear-indexes-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Crear indexes</a></span></li><li><span><a href="#Empezamos-con-las-Geoqueries" data-toc-modified-id="Empezamos-con-las-Geoqueries-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Empezamos con las Geoqueries</a></span></li><li><span><a href="#Intersects" data-toc-modified-id="Intersects-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Intersects</a></span></li><li><span><a href="#GeoWithin" data-toc-modified-id="GeoWithin-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>GeoWithin</a></span></li><li><span><a href="#Near" data-toc-modified-id="Near-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Near</a></span></li><li><span><a href="#Probamos-Geoqueries-con-datos-reales" data-toc-modified-id="Probamos-Geoqueries-con-datos-reales-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Probamos Geoqueries con datos reales</a></span></li><li><span><a href="#Vamos-a-buscar-las-librerías-que-están-cerca-de-mi-casa" data-toc-modified-id="Vamos-a-buscar-las-librerías-que-están-cerca-de-mi-casa-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Vamos a buscar las librerías que están cerca de mi casa</a></span></li><li><span><a href="#Vamos-a-buscar-si-esas-librerías-que-hay-a-1km-a-la-redonda-de-la-casa-de-Ras,-están-todas-en-el-mismo-distrito" data-toc-modified-id="Vamos-a-buscar-si-esas-librerías-que-hay-a-1km-a-la-redonda-de-la-casa-de-Ras,-están-todas-en-el-mismo-distrito-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Vamos a buscar si esas librerías que hay a 1km a la redonda de la casa de Ras, están todas en el mismo distrito</a></span></li></ul></div>

## ¿Qué podemos hacer con las geoqueries?
Con las geoquerías podemos responder a preguntas como las siguientes
 * ¿Dónde están las farmacias más cercanas a mi ubicación?
 * ¿Qué restaurantes hay en esta poligonal?

Cada punto de la Tierra se caracteriza por dos números:
 * Longitud: ángulo respecto al meridiano de Greenwich. Va de -180° (antiGreenwich) a +180° (también antiGreenwich)
 * Latitud: ángulo con respecto al ecuador. Va de -90° (sur) a +90° (norte)

![latierra](../images/latlon.gif)

## Generamos la conexión con Mongo

In [2]:
from pymongo import MongoClient

In [5]:
client = MongoClient("localhost:27017")
db = client.get_database("ironhack")
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack')

## Importar colecciones     
Sigamos la documentación oficial de Mongo: [Mongo Geo Example](https://docs.mongodb.com/manual/tutorial/geospatial-tutorial/)
Creamos estas dos colecciones nuevas:
 * `restaurants2`
 * `neighborhoods`

In [6]:
rest = db.get_collection("rest")
barr = db.get_collection("barrios")

## Crear indexes
La indexación geoespacial de MongoDB permite ejecutar eficientemente **consultas espaciales** sobre una colección que contiene formas y puntos geoespaciales.
Vamos a crear un índice geográfico para que Mongo sepa que queremos hacer geo-consultas en esta colección.  
Esto sólo debe hacerse una vez, y ahora la colección de Mongo está correctamente indexada para siempre.       
[Documentación](https://docs.mongodb.com/manual/geospatial-queries/#geospatial-indexes) de los índices.

In [7]:
from pymongo import GEOSPHERE

In [9]:
# Creamos el  index 2dsphere desde pymongo
db.barrios.create_index([("geometry", GEOSPHERE)])

'geometry_2dsphere'

Es importante indicarle el campo donde tenemos tanto el tipo de elemento (polígonos o  puntos (point)) como las coordenadas.

In [10]:
"""
Si quiero crear el de restaurants LE TENGO QUE DECIR CUÁL ES EL CAMPO DÓNDE ESTÁ LA info no solo de las
coordenadas si no del tipo de dato que es, polígono, multipolígono....en este caso está dentro del campo location
en la colección de barrios está dentro de un campo llamado geometry
"""
db.rest.create_index([("location", GEOSPHERE)])

'location_2dsphere'

## Empezamos con las Geoqueries   
¿En qué barrio estoy?      
¿Qué elementos **intersecan** el elemento dado?

## Intersects

Necesitamos tener los datos  con tipo polígono en este caso tenemos una colección con barrios donde cada barrio  tiene dentro  el polígono que lo delimita. Y cada barrio es un documento, por eso cuando  intersecta me devuelve el documento y yo veo  que es el barrio  en concreto.    
Selecciona los documentos cuyos datos geoespaciales se cruzan con un objeto GeoJSON especificado; es decir, cuando la intersección de los datos y el objeto especificado no está vacía.

In [11]:
def type_point(lista):
    return {"type":"Point", "coordinates": lista}

In [12]:
coordenadas = [-73.93, 40.82]

In [13]:
barr.find_one().keys()

dict_keys(['_id', 'geometry', 'name'])

El geometry que va sin dolar es porque vamos a buscar en la colección de barrios y donde está la información es dentro de la key geometry, se ve justo aquí arriba 👆🏻. El geometry que lleva dolar es la sintaxis de mongo para todas las queries geoespaciales. 
Para hacer una query geoespacial a Mongo tengo que pasarle las coordenadas en tipo point

In [14]:
punto = type_point(coordenadas)
punto

{'type': 'Point', 'coordinates': [-73.93, 40.82]}

In [16]:
proy = {"_id":0, "name":1}

In [17]:
barr.find_one({"geometry": {"$geoIntersects": {"$geometry": punto}}}, proy)

{'name': 'West Concourse'}

In [38]:
barr.find_one().keys()

dict_keys(['_id', 'geometry', 'name'])

In [21]:
# LO QUE LLAMÁBAMOS FILTRO A LA HORA DE HACER LA QUERY
filtro = {"geometry": {"$geoIntersects": {"$geometry": punto}}} #punto está en tipo point

In [22]:
barr.find_one(filtro,proy)

{'name': 'West Concourse'}

## GeoWithin

¿Qué elementos están **contenidos** en el elemento dado?     
¿Qué restaurantes hay en este barrio (polígono)?

In [25]:
# Vemos la estructura de los documentos sacando uno y viendo sus keys
barr.find_one().keys()

dict_keys(['_id', 'geometry', 'name'])

In [33]:
#bUSCAMOS UN barrio en  concreto
barrio = barr.find_one({"name": "Bedford"})
#barrio

In [36]:
#Extraigo las coordenadas
coord_barrio = barrio["geometry"]
#coord_barrio

In [37]:
#Saco las keys de un documento de la coleccción  de restaurantes
rest.find_one().keys()

dict_keys(['_id', 'location', 'name'])

In [40]:
# ER FIRTRO
#Location porque la key de la colección donde están las coordenadas se llama location
query = {"location": {"$geoWithin": {"$geometry": coord_barrio}}}

In [41]:
restaurantes_bedford = list(rest.find(query))

In [44]:
restaurantes_bedford[0:2]

[{'_id': ObjectId('55cba2486c522cafdb0592d3'),
  'location': {'coordinates': [-73.9467295, 40.6804619], 'type': 'Point'},
  'name': 'Island Salad'},
 {'_id': ObjectId('55cba2476c522cafdb056c31'),
  'location': {'coordinates': [-73.94659899999999, 40.6804119],
   'type': 'Point'},
  'name': 'Tak King Chinese Restaurant'}]

In [46]:
import pandas as pd
df = pd.DataFrame(restaurantes_bedford)
df.head()

,_id,location,name
0,55cba2486c522cafdb0592d3,"{'coordinates': [-73.9467295, 40.6804619], 'ty...",Island Salad
1,55cba2476c522cafdb056c31,"{'coordinates': [-73.94659899999999, 40.680411...",Tak King Chinese Restaurant
2,55cba2476c522cafdb055380,"{'coordinates': [-73.94650109999999, 40.680409...",Tastee Pattee
3,55cba2476c522cafdb0543fd,"{'coordinates': [-73.9459945, 40.6804254], 'ty...",Burger King
4,55cba2476c522cafdb0565ee,"{'coordinates': [-73.9455406, 40.6803509], 'ty...",Wing Chang Food House


## Near

¿Qué elementos se encuentran a una distancia de una geometría determinada?     
`$nearSphere`/`$near` y `$maxDistance` nos ayudan a encontrar todos los elementos dentro de `maxDistance` metros de la geometría deseada, **ordenados** de más cercano a más lejano.

In [47]:
#  Mi posición, voy a buscar restaurantes cercanos a  ESTE punto
coordenadas2 = [-73.93, 40.82]

In [48]:
# Km que quiero  caminar
metros = 300

In [50]:
coord = type_point(coordenadas2)
coord

{'type': 'Point', 'coordinates': [-73.93, 40.82]}

In [51]:
query = {"location": {"$near": {"$geometry": coord, "$maxDistance": metros}}}

In [54]:
list(rest.find(query))[0:3]

[{'_id': ObjectId('55cba2486c522cafdb059dca'),
  'location': {'coordinates': [-73.92961799999999, 40.81970099999999],
   'type': 'Point'},
  'name': ''},
 {'_id': ObjectId('55cba2476c522cafdb056b6f'),
  'location': {'coordinates': [-73.93032130000002, 40.8193837],
   'type': 'Point'},
  'name': 'Subway'},
 {'_id': ObjectId('55cba2476c522cafdb05517e'),
  'location': {'coordinates': [-73.92856720000002, 40.8190635],
   'type': 'Point'},
  'name': 'Glacken Bar'}]

In [58]:
barrio_cerca = {"geometry": {"$near": {"$geometry": coord, "$maxDistance": metros}}}

In [61]:
list(barr.find(barrio_cerca, proy))

[{'name': 'West Concourse'},
 {'name': 'East Concourse-Concourse Village'},
 {'name': 'Mott Haven-Port Morris'}]

¿En qué barrio está este punto?

In [63]:
barr.find_one({"geometry": {"$geoIntersects": {"$geometry": coord}}},proy)

{'name': 'West Concourse'}

## Probamos Geoqueries con datos reales

Me he bajado un geojson con los distritos de Madrid de [esta página](https://team.carto.com/u/jsanz/tables/distritos/public) y los voy a cargar en mi base de datos de Mongo

In [66]:
import json
with open ("distritos.geojson") as f: #Abro el geojson de distritos para explorarlo en python
    distritos = json.load(f)

In [69]:
distritos.keys()

dict_keys(['type', 'features'])

Veo que el value de "Features" es la lista de diccionarios donde están los polígonos. Cada diccionario es un distrito. Así que extraigo features y lo cargo en un json llamado distritos_bien

In [72]:
#distritos["features"] 

In [71]:
with open ("distritos_bien.json", "w") as archivo:
    json.dump(distritos["features"], archivo)

Me vuelvo a traer la función geocode que me daba la latitud y la longitud de un string directamente en tipo point 

In [86]:
import requests
def geocode(direccion):
    """
    Esta función saca las coordenadas de la dirección que le pases
    """
    data = requests.get(f"https://geocode.xyz/{direccion}?json=1").json()
    try:
        return {"type": "Point", "coordinates": [float(data["latt"]), float(data["longt"])]}
    except:
        return data

Me saco el type point de Ironhack para buscar en qué distrito está (voy a buscar en mi propia colección de distritos de Madrid)

In [94]:
ironhack = geocode("Paseo de la chopera, 14, Madrid")

In [191]:
ironhack

{'type': 'Point', 'coordinates': [40.39652, -3.7011]}

Resulta que mis polígonos en el gejson que me he descargado tienen la latitud y la longitud al revés, así que voy a darle la vuelta para hacer la query

In [113]:
ironhack_alreves = type_point([-3.7011,40.39652])

In [157]:
ironhack_alreves

{'type': 'Point', 'coordinates': [-3.7011, 40.39652]}

In [114]:
# Preparo la query ....
query_barrio_ironhack = {"geometry": {"$geoIntersects": {"$geometry": ironhack_alreves}}}

In [192]:
# Me conecto a la colección de distritos
dis = db.get_collection("distritos")

In [98]:
# Busco uno para probar que funciona
dis.find_one()

In [118]:
# Me guardo la proyección para que solo me dé el nombre del distrito
proy_ = {"properties.nombre": 1, "_id":0}

In [120]:
# Busco en la base de datos con la query que he preparado arriba y la proyección
dis.find_one(query_barrio_ironhack, proy_)

{'properties': {'nombre': 'Arganzuela'}}

Efectivamente, Ironhack está en el barrio de Arganzuela. Bravo, fantasía 🚀🗺🌈

## Vamos a buscar las librerías que están cerca de mi casa

In [135]:
# Utilizamos la función de geocode para que me dé la Latitud y la Longitud, como antes con ironhack
ras = geocode("Plaza de Cascorro 8, Madrid")

In [136]:
ras

{'type': 'Point', 'coordinates': [40.41029, -3.70713]}

In [184]:
# Me conecto a la colección de librerías de mi basde datos
lib = db.get_collection("librerías")

In [183]:
# Preparo la query, voy a buscar en la colección de librerías las que están cerca de mi barrio
librerias = {"location": {"$near": {"$geometry": ras, "$maxDistance": 1000}}}

In [185]:
# Hago la query y me devuelve una lista
libres_cerca = list(lib.find(librerias))

In [186]:
# Exploro la primera para ver el resultado
libres_cerca[0]

{'_id': ObjectId('618ce0b35de447ecd07221cc'),
 'nombre': 'Olor de la Lluvia Librería',
 'latitud': 40.41073,
 'longitud': -3.710335,
 'location': {'type': 'Point', 'coordinates': [40.41073, -3.710335]}}

## Vamos a buscar si esas librerías que hay a 1km a la redonda de la casa de Ras, están todas en el mismo distrito

In [187]:
# Recordamos que tenemos que darle la vuelta a latitud,longitud
libres_cerca[0]["location"]["coordinates"][::-1]

[-3.710335, 40.41073]

In [188]:
total = [] # lista para resultado final
for libreria in libres_cerca: #iteramos por la lista entera de librerías de antes 
    point = type_point([libreria["longitud"],libreria["latitud"]]) #preparamos tipo point de cada una 
    query = {"geometry": {"$geoIntersects": {"$geometry": point}}} #hacemos la query para buscar el barrio 
    resultado = dis.find_one(query) # hacemos la query 
    libre = {libreria["nombre"]:resultado["properties"]["nombre"]} # nombre de la librería y nombre del distrito de la query
    total.append(libre)

In [189]:
print(total)

[{'Olor de la Lluvia Librería': 'Centro'}, {'Librería LaMalatesta': 'Centro'}, {'Librería Mujeres': 'Centro'}, {'Librería Jímenez': 'Centro'}, {'Librería El Corte Inglés': 'Centro'}, {'Librería 8 1/2': 'Centro'}]
